# Backtrader testing

## Momentun

* RSI > 70 overbought, < 30 oversold

## Volatility

* BB - %b > 1 sell, < 0, buy 


## Reference

* https://towardsdatascience.com/trading-strategy-technical-analysis-with-python-ta-lib-3ce9d6ce5614
* https://towardsdatascience.com/efficient-frontier-optimize-portfolio-with-scipy-57456428323e
* https://towardsdatascience.com/trading-strategy-back-testing-with-backtrader-6c173f29e37f


In [7]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import backtrader as bt
import seaborn as sns

In [ ]:
bt.feeds.